# Derivation of equation of motion
Started in class, on 8-Mar-2019 by T. Fitzgerald

Modified on 18-Mar-2019 to include an external force.

A sketch of the system model is
![alt text](sketch-of-system.svg "System diagram")

The goal is to find the equation of motion by using Lagrange's equation.  Along the way several fun facts about using SymPy/Julia are discussed.

In [1]:
using Pkg
Pkg.activate(".")
# Pkg.add("SymPy") # this is only needed the first time
using SymPy

Define the parameters of the system

In [2]:
L1, L2, L3, k1, k2, c1, c2, m, JG, g = symbols("L1, L2, L3, k1, k2, c1, c2, m, J_G, g", positive=true)
;

Define the coordinates of the system $x,\theta$ as a `SymFunction`

In [3]:
t = symbols("t")
y = SymFunction("y")(t)
θ = SymFunction("theta")(t)
f = SymFunction("f")(t)
;

## Building $T$, $V$, and $D$

I'll start by building the displacements of each tip, and then construct the system's potential energy

In [4]:
y1 = y - L1*sin(θ)
y2 = y + L2*sin(θ)
V = 1//2*k1*y1^2 + 1//2*k2*y2^2 + m*g*y

                                        2                               2
           k1*(-L1*sin(theta(t)) + y(t))    k2*(L2*sin(theta(t)) + y(t)) 
g*m*y(t) + ------------------------------ + -----------------------------
                         2                                2              

Next is Rayleigh's dissipation function for the viscous dampers

In [5]:
D = 1//2*c1*diff(y1,t)^2 + 1//2*c2*diff(y2,t)^2

                                               2                              
   /                   d              d       \       /                 d     
c1*|- L1*cos(theta(t))*--(theta(t)) + --(y(t))|    c2*|L2*cos(theta(t))*--(the
   \                   dt             dt      /       \                 dt    
------------------------------------------------ + ---------------------------
                       2                                                 2    

                  2
         d       \ 
ta(t)) + --(y(t))| 
         dt      / 
-------------------
                   

Last is the system's kinetic energy.  In general for a planar rigid body this is
    $$ T = \frac{1}{2} m_G \dot{\vec{r}} \cdot \dot{\vec{r}} + \frac{1}{2}J_G \omega^2 $$

In [6]:
T = 1//2*m*diff(y,t)^2 + 1//2*JG*diff(θ,t)^2

                  2               2
    /d           \      /d       \ 
J_G*|--(theta(t))|    m*|--(y(t))| 
    \dt          /      \dt      / 
------------------- + -------------
         2                  2      

## Generalized forces

In [7]:
rf = y*[0,1] + L3*(cos(θ)*[1,0] + sin(θ)*[0,1])

2-element Array{Sym,1}:
        L3*cos(theta(t))
 L3*sin(theta(t)) + y(t)

In [8]:
F = f*[0,1]

2-element Array{Sym,1}:
    0
 f(t)

In [9]:
Qy = transpose(F)*diff(rf,y)

f(t)

In [10]:
Qθ = transpose(F)*diff(rf,θ)

L3*f(t)*cos(theta(t))

This looks very familiar.  $Q_y$ is just the force, and $Q_\theta$ is the moment about the center of mass.

## Lagrange's equation
Next, we'll build Lagrange's equations for the system with external forces
$$ \frac{d}{dt}\left(\frac{\partial L}{\partial \dot{y}}  \right) - \frac{\partial L}{\partial y} + \frac{\partial D}{\partial \dot{y}}  = Q_y$$
$$ \frac{d}{dt}\left(\frac{\partial L}{\partial \dot{\theta}}  \right) - \frac{\partial L}{\partial \theta} + \frac{\partial D}{\partial \dot{\theta}}  = Q_\theta$$

In [11]:
L = T - V |> simplify

                  2                                                           
    /d           \                                                            
J_G*|--(theta(t))|                                           2                
    \dt          /               k1*(L1*sin(theta(t)) - y(t))    k2*(L2*sin(th
------------------- - g*m*y(t) - ----------------------------- - -------------
         2                                     2                              

                               2
                     /d       \ 
               2   m*|--(y(t))| 
eta(t)) + y(t))      \dt      / 
---------------- + -------------
 2                       2      

I can directly just plug away since SymPy can differential with respect to the derivative of a `SymFunction`

In [12]:
diff(L, diff(y,t), t ) - diff(L, y) + diff(D, diff(y,t)) - Qy

   /                     d                d       \      /                   d
c1*|- 2*L1*cos(theta(t))*--(theta(t)) + 2*--(y(t))|   c2*|2*L2*cos(theta(t))*-
   \                     dt               dt      /      \                   d
--------------------------------------------------- + ------------------------
                         2                                                    
                                                                              

                d       \                                                     
-(theta(t)) + 2*--(y(t))|                                                     
t               dt      /         k1*(-2*L1*sin(theta(t)) + 2*y(t))   k2*(2*L2
------------------------- + g*m + --------------------------------- + --------
2                                                 2                           
                                                                              

                                             
    

### Elegance
I can also be a little more elegant since I know I have two of these equations to build.  I can use a list comprehension

In [13]:
# List comprehension
[ i+1 for i in 1:3 ]

3-element Array{Int64,1}:
 2
 3
 4

In [14]:
[ diff(L, diff(q,t), t) - diff(L,q) + diff(D,diff(q,t)) |> simplify for q in [y,θ]  ] - [Qy, Qθ]

2-element Array{Sym,1}:
                                                                                     -c1*(L1*cos(theta(t))*Derivative(theta(t), t) - Derivative(y(t), t)) + c2*(L2*cos(theta(t))*Derivative(theta(t), t) + Derivative(y(t), t)) + g*m - k1*(L1*sin(theta(t)) - y(t)) + k2*(L2*sin(theta(t)) + y(t)) + m*Derivative(y(t), (t, 2)) - f(t)
 J_G*Derivative(theta(t), (t, 2)) + L1*c1*(L1*cos(theta(t))*Derivative(theta(t), t) - Derivative(y(t), t))*cos(theta(t)) + L1*k1*(L1*sin(theta(t)) - y(t))*cos(theta(t)) + L2*c2*(L2*cos(theta(t))*Derivative(theta(t), t) + Derivative(y(t), t))*cos(theta(t)) + L2*k2*(L2*sin(theta(t)) + y(t))*cos(theta(t)) - L3*f(t)*cos(theta(t))

Or (as Camp pointed out) we can also define a function, and then broadcast it onto the coordinates

In [15]:
Lag(q) = diff(L, diff(q,t), t) - diff(L,q) + diff(D,diff(q,t)) - transpose(F)*diff(rf,q) |> simplify
[ Lag(q) for q in [y,θ] ]

2-element Array{Sym,1}:
                                                                                     -c1*(L1*cos(theta(t))*Derivative(theta(t), t) - Derivative(y(t), t)) + c2*(L2*cos(theta(t))*Derivative(theta(t), t) + Derivative(y(t), t)) + g*m - k1*(L1*sin(theta(t)) - y(t)) + k2*(L2*sin(theta(t)) + y(t)) + m*Derivative(y(t), (t, 2)) - f(t)
 J_G*Derivative(theta(t), (t, 2)) + L1*c1*(L1*cos(theta(t))*Derivative(theta(t), t) - Derivative(y(t), t))*cos(theta(t)) + L1*k1*(L1*sin(theta(t)) - y(t))*cos(theta(t)) + L2*c2*(L2*cos(theta(t))*Derivative(theta(t), t) + Derivative(y(t), t))*cos(theta(t)) + L2*k2*(L2*sin(theta(t)) + y(t))*cos(theta(t)) - L3*f(t)*cos(theta(t))

In [16]:
eom = Lag.([y,θ])

2-element Array{Sym,1}:
                                                                                     -c1*(L1*cos(theta(t))*Derivative(theta(t), t) - Derivative(y(t), t)) + c2*(L2*cos(theta(t))*Derivative(theta(t), t) + Derivative(y(t), t)) + g*m - k1*(L1*sin(theta(t)) - y(t)) + k2*(L2*sin(theta(t)) + y(t)) + m*Derivative(y(t), (t, 2)) - f(t)
 J_G*Derivative(theta(t), (t, 2)) + L1*c1*(L1*cos(theta(t))*Derivative(theta(t), t) - Derivative(y(t), t))*cos(theta(t)) + L1*k1*(L1*sin(theta(t)) - y(t))*cos(theta(t)) + L2*c2*(L2*cos(theta(t))*Derivative(theta(t), t) + Derivative(y(t), t))*cos(theta(t)) + L2*k2*(L2*sin(theta(t)) + y(t))*cos(theta(t)) - L3*f(t)*cos(theta(t))

That last way looks really slick to me, I'm going to remember that one.

## What do we likely need to do now?
It's a really common problem to derive a system model, and then need to simulate it.  As we can likely recall, standard ode solvers (like Matlab's `ode45`) require first order form.  So if we need to simulate this system of equations, we probably need to put it into first order form.  To do this, I'll first solve it explicity for the second derivatives

In [17]:
sol = solve( eom, diff([y,θ],t,t) )

Dict{Sym,Sym} with 2 entries:
  Derivative(y(t), (t, 2))     => (L1*c1*cos(theta(t))*Derivative(theta(t), t) …
  Derivative(theta(t), (t, 2)) => (-L1^2*c1*cos(theta(t))*Derivative(theta(t), …

Next, I'll define some states that I want to transform these equations into.
$$z = [ y \quad \theta \quad \dot y \quad \dot \theta]^T$$

In [18]:
z1,z2,z3,z4, ff = symbols("z1,z2,z3,z4, ff")
rule1 = Dict( diff(y,t)=> z3, diff(θ,t)=>z4  )
rule2 = Dict( y=> z1, θ=>z2 )

Dict{Sym,Sym} with 2 entries:
  theta(t) => z2
  y(t)     => z1

In [19]:
rule1[diff(y,t)]

z3

In [20]:
eq3 = sol[diff(θ,t,t)] |> subs(rule1) |> subs(rule2)

/    2                   2                                      2             
\- L1 *c1*z4*cos(z2) - L1 *k1*sin(z2) + L1*c1*z3 + L1*k1*z1 - L2 *c2*z4*cos(z2
------------------------------------------------------------------------------
                                                                   J_G        

      2                                           \        
) - L2 *k2*sin(z2) - L2*c2*z3 - L2*k2*z1 + L3*f(t)/*cos(z2)
-----------------------------------------------------------
                                                           

To get the entire set of state space equations, I can do the following.  I'll make a list of the rates of the states (think left-side of the state-space equation $\dot z = f(z)$, and then sub-in in terms of the rules I've defined

In [21]:
Fss = diff( [y, θ, diff(y, t), diff(θ,t)], t) |> subs(sol) |> subs(rule1) |> subs(rule2)

4-element Array{Sym,1}:
                                                                                                                                               z3
                                                                                                                                               z4
                             (L1*c1*z4*cos(z2) + L1*k1*sin(z2) - L2*c2*z4*cos(z2) - L2*k2*sin(z2) - c1*z3 - c2*z3 - g*m - k1*z1 - k2*z1 + f(t))/m
 (-L1^2*c1*z4*cos(z2) - L1^2*k1*sin(z2) + L1*c1*z3 + L1*k1*z1 - L2^2*c2*z4*cos(z2) - L2^2*k2*sin(z2) - L2*c2*z3 - L2*k2*z1 + L3*f(t))*cos(z2)/J_G

### Leaving this notebook

If we are deriving this equation for some larger purpose, it is likely we'll need to write these equations down somewhere else.  We might even want to use a different program altogether and jumpt to Matlab.  We have some options here, and they revolve around reaching into SymPy a bit farther (not using an exposed function call) to re-print the expression in another langage.  Depending on the language we may need to change the SymFunction `f(t)` into a regular variable, as it may not translate.

- If we want to move to Matlab, we need the `:octave_code` routine.  [GNU Octave](https://www.gnu.org/software/octave/) is an open source Matlab-like program.
- If we want to move to Julia, we need the `:julia_code` routine.
- There are many others, including C `:ccode` and Fortran `:fcode`.

In [22]:
sympy_meth(:octave_code, subs( eq3, f=>ff) ) 

"(-L1.^2.*c1.*z4.*cos(z2) - L1.^2.*k1.*sin(z2) + L1.*c1.*z3 + L1.*k1.*z1 - L2.^2.*c2.*z4.*cos(z2) - L2.^2.*k2.*sin(z2) - L2.*c2.*z3 - L2.*k2.*z1 + L3.*ff).*cos(z2)./J_G"

or 

In [23]:
sympy_meth(:octave_code, subs(Fss, f=>ff) )

"[z3; z4; (L1.*c1.*z4.*cos(z2) + L1.*k1.*sin(z2) - L2.*c2.*z4.*cos(z2) - L2.*k2.*sin(z2) - c1.*z3 - c2.*z3 + ff - g.*m - k1.*z1 - k2.*z1)./m; (-L1.^2.*c1.*z4.*cos(z2) - L1.^2.*k1.*sin(z2) + L1.*c1.*z3 + L1.*k1.*z1 - L2.^2.*c2.*z4.*cos(z2) - L2.^2.*k2.*sin(z2) - L2.*c2.*z3 - L2.*k2.*z1 + L3.*ff).*cos(z2)./J_G]"

If we want to get rid of the quotes and other string oddities, we can print this string using `println` and then copy-paste what we need.

In [24]:
sympy_meth(:octave_code, subs( eq3, f=>ff)) |> println

(-L1.^2.*c1.*z4.*cos(z2) - L1.^2.*k1.*sin(z2) + L1.*c1.*z3 + L1.*k1.*z1 - L2.^2.*c2.*z4.*cos(z2) - L2.^2.*k2.*sin(z2) - L2.*c2.*z3 - L2.*k2.*z1 + L3.*ff).*cos(z2)./J_G


### Alternatives
We can also use the `lambdify` function that directly generates a Julia function from an expression.  This is useful if we want to do a bunch of computations on these functions without leaving or re-writing the equations.

In [25]:
?lambdify

search: lambdify lambdify_expr



`lambidfy(ex::Sym,[vars]; typ=Any)`: Lambidfy an expression returning a native Julia function.  SymPy's [lambdify](http://docs.sympy.org/dev/modules/utilities/lambdify.html) function translates code into Python, this translates an expression into a `Julia` function.

Evaluating the function does not call into SymPy, so should be much faster.

The optional `[vars]` specifies the order of the variables when more than one is in `ex`. The default is to use the ordering of `free_symbols(ex)`.

The keyword aruguments allow for the passing of expressions that are not covered by the default ones. These are dictionaries whose keys are strings with a SymPy name and whose values are symbols representing `Julia` values. For examples `Dict("sin"=>:sin)` could be used to map a function, were that not already done.

Additionally, a DataType keyword can be specified for the function.

Not all expressions can be lambdified. If not, an error is thrown.

There are two steps: a conversion from SymPy code to an expression, then a conversion into an anonymous function. The first step is performed either through `walk_expression` (the default) or through a converter provided by SymPy, `SymPy.sympy_meth(:julia_code, expr)`. The latter has some issues with the dot notation so is not the default. To use it, pass `do_julia_code=true` to `lambdify`.

Some simple examples

```
@vars x y
lambdify(x^2)(2)       # 4
lambdify(x*y^2)(2,3)   # 2*3^2 using default ordering
lambdify(x*y^2, [y, x])(2,3) # 3*2^2, as function is (y,x) -> x*y^2 equivalent in Julia
```

Compare times

```
xs = rand(1000)
@vars x
ex = sin(x)*cos(2x) * exp(x^2/2)
map(u -> N(ex(u)), xs)   # 3.435850 seconds
map(lambdify(ex), xs)    # 0.007085 seconds
```


In [26]:
epr = m+1

m + 1

In [27]:
Epr(m) = lambdify(epr)(m)

Epr (generic function with 1 method)

In [28]:
Epr(1)

2